In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [ ]:
config = load_config()
od = '../../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

# Goal: get all exons that are detected in one population that aren't in the other

## novelty and tid info

In [ ]:
df = get_internal_exon_info(proc_cfg(config['lr']['gtf_filt_with_genes'], od),
                            proc_cfg(config['ref']['gtf'], od))

## population detection parsing

In [4]:
# first, get all transcripts that are specific b/w 2 population
mt_df = pd.read_csv('../../data/05_mastertable/29102024_PODER_mastertable.tsv', sep='\t')
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)
print(len(mt_df.loc[mt_df.pop_spec_t==True, 'isoform'].unique()))

2267


In [13]:
t_df = mt_df[['isoform', 'AJI', 'CEU',
              'HAC', 'ITU', 'LWK',
              'MPC', 'PEL', 'YRI']]
t_df.head()

,isoform,AJI,CEU,HAC,ITU,LWK,MPC,PEL,YRI
0,ENST00000741425.1,1,0,0,0,0,0,0,0
1,transcript_0,1,0,0,0,0,0,0,0
2,transcript_1,1,0,0,0,0,0,1,0
3,transcript_10,1,1,0,1,0,0,0,0
4,transcript_11,2,1,0,1,0,0,2,0


,Unnamed: 0,Chromosome,Strand,Start,End,eid,Strand_gc,Start_gc,End_gc,eid_gc,Overlap
0,0,GL000008.2,+,83926,84014,GL000008.2_+_83926_84014_,+,83859,84014,GL000008.2_+_83859_84014_,88
1,1,GL000008.2,+,83926,84014,GL000008.2_+_83926_84014_,+,83926,84145,GL000008.2_+_83926_84145_,88
2,2,GL000008.2,+,83926,84014,GL000008.2_+_83926_84014_,+,83926,84014,GL000008.2_+_83926_84014_,88
3,3,GL000008.2,+,83926,84014,GL000008.2_+_83926_84014_,+,83952,84145,GL000008.2_+_83952_84145_,62
4,4,GL000008.2,+,85456,85477,GL000008.2_+_85456_85477_,+,85442,85477,GL000008.2_+_85442_85477_,21
